In [1]:
import mysql.connector as connection
import pandas as pd
import numpy as np
import pymysql

In [2]:
# PARA CONECTAR CUALQUIER BD RELACIONALES
usuario = 'root'        # completar usuario (dato Workbench)
contra = 'root'         # completar contraseña  (dato Workbench)
dominio = 'localhost'   # completar dominio (dato Workbench)
puerto = '3306'         # completar puerto (dato Workbench)
db = 'academia'

In [3]:
conn = pymysql.connect(host = dominio, user = usuario, passwd = contra, db = db)

In [4]:
cur = conn.cursor()

        -- Nota media de cada asignatura.

In [5]:
cur.execute('''SELECT nombre, 
                    (SELECT AVG(nota) FROM asig_alumno AS aa WHERE aa.asignatura_id = a.asignatura_id) AS nota_media
	            FROM asignatura AS a
                GROUP BY nombre;''')

notamedia_asig = cur.fetchall()
# print(notamedia_asig)

pd.DataFrame(notamedia_asig, columns=['asignatura', 'nota_media'])

,asignatura,nota_media
0,LENGUA,5.0000
1,MATEMATICAS,5.1500
2,FILOSOFIA,5.4500
3,HISTORIA,4.7000
4,PLASTICA,5.3000
5,CIENCIAS NATURALES,4.9500


        -- Nota media asociada a cada profesor (media las notas de sus asignaturas)

In [6]:
cur.execute('''SELECT profesor_id,
		                (SELECT nombre FROM profesor AS p WHERE p.profesor_id = ap.profesor_id) AS nombre,
                        (SELECT apellido FROM profesor AS p WHERE p.profesor_id = ap.profesor_id) AS apellido,
                        (SELECT AVG(nota) FROM asig_alumno AS aa WHERE aa.asignatura_id = ap.asignatura_id) AS nota_media
	            FROM asig_profesor AS ap
                GROUP BY profesor_id, nombre, apellido
                ORDER BY profesor_id;''')

notamedia_prof = cur.fetchall()
# print(notamedia_prof)

pd.DataFrame(notamedia_prof, columns=['profesor_id', 'nombre_prof', 'apellido_prof', 'nota_media'])

,profesor_id,nombre_prof,apellido_prof,nota_media
0,1,Paula,Lopez Alonso,4.9500
1,2,Alejandro,Lopez Alonso,5.1500
2,9,Almudena,Gomez de Robledo,4.7000
3,11,Alejandro,Roble del Amo,5.0000
4,12,Alejandro,Diaz Serrano,5.1500
5,13,Paula,Gomez de Robledo,5.3000
6,14,Alejandro,Diaz Serrano,4.9500
7,16,Fernando,Diaz Serrano,5.4500
8,17,Pablo,Lopez Alonso,5.3000
9,18,Alejandro,Gomez de Robledo,5.0000


        -- ¿Cuál es la nota del último examen realizado por el alumno con mayor nota media?

In [7]:
cur.execute('''SELECT max(nota),
		                alumno_id, 
		                (SELECT nombre FROM alumno AS alum WHERE alum.alumno_id = aa.alumno_id) AS nombre,
                        (SELECT apellido FROM alumno AS alum WHERE alum.alumno_id = aa.alumno_id) AS apellido
	            FROM asig_alumno AS aa
                WHERE aa.alumno_id = (SELECT aa.alumno_id FROM asig_alumno AS aa GROUP BY alumno_id ORDER BY AVG(nota) DESC LIMIT 1)
				            AND aa.asignatura_id = (SELECT aa.asignatura_id FROM asignatura AS a WHERE a.asignatura_id =
																				(SELECT e.asignatura_id FROM examen AS e ORDER BY e.fecha_examen DESC LIMIT 1));''')
                                                                    
ultimanota_media = cur.fetchall()
# print(ultimanota_media)

pd.DataFrame(ultimanota_media, columns=['nota', 'alumno_id', 'nombre_alumno', 'apellido_alumno'])

,nota,alumno_id,nombre_alumno,apellido_alumno
0,10,7,José Ignacio,Ruiz vasquez


        -- ¿Cuál es la asignatura más dificil?

In [8]:
cur.execute('''SELECT AVG(nota),
		            asignatura_id,
                    (SELECT nombre FROM asignatura AS a WHERE a.asignatura_id = aa.asignatura_id) AS asignatura 
	            FROM asig_alumno AS aa 
	            GROUP BY asignatura_id
                ORDER BY AVG(nota) ASC LIMIT 1;''')

asig_dificil = cur.fetchall()
# print(asig_dificil)

pd.DataFrame(asig_dificil, columns=['nota_media', 'asignatura_id', 'asignatura'])

,nota_media,asignatura_id,asignatura
0,4.7000,4,HISTORIA


        -- Busca el grupo de riesgo de cada asignatura (alumnos que peor van) y verifica si hay alguno que esté en todas.

In [9]:
cur.execute('''SELECT asignatura_id
	            FROM asignatura;''')

asignaturas = cur.fetchall()
# print(asignaturas)

asignaturas = np.array(asignaturas)
asignaturas = asignaturas.flatten().tolist()
asignaturas

[1, 2, 3, 4, 5, 6]

In [10]:
grupo_riesgo = list()
for asig_id in asignaturas:
    cur.execute(f'''SELECT alumno_id
	            FROM asig_alumno
                WHERE asignatura_id = {asig_id}
                ORDER BY nota ASC LIMIT 5;''')

    asig_riesgo = cur.fetchall()
    # print(asig_riesgo)
    grupo_riesgo.append(asig_riesgo)

grupo_riesgo = np.array(grupo_riesgo).reshape(len(asignaturas), 5)
grupo_riesgo

array([[69, 35, 89, 39, 57],
       [38, 29, 58, 80, 94],
       [70, 23, 31, 73, 38],
       [30, 74, 50, 76, 91],
       [73,  1, 50, 13, 23],
       [23, 55, 79, 71, 75]])

In [11]:
alumno_repet = dict()

for asig in range(len(asignaturas)):
    for alumn in grupo_riesgo[asig]:
        if alumn not in alumno_repet:
            alumno_repet[alumn] = 0
        if alumn in grupo_riesgo:
            alumno_repet[alumn] += 1
         
alumno_repet = sorted(alumno_repet.items(), key=lambda item:item[1], reverse=True)

pd.DataFrame(alumno_repet, columns=['alumno_id', 'cant_asignaturas'])

,alumno_id,cant_asignaturas
0,23,3
1,38,2
2,73,2
3,50,2
4,69,1
5,35,1
6,89,1
7,39,1
8,57,1
9,29,1
